In [1]:
from impact2_engine.Enrolment import Enrolment
import pandas as pd
import yaml
from dateutil.relativedelta import relativedelta, MO

/Users/robert_pinsight/opt/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
with open('../../impact2_engine/config/config.yaml', 'r', encoding = 'utf-8') as safety_stream:
    enrolment_config = yaml.safe_load(safety_stream)

In [3]:
enrolment_config

{'data_path': 'enroll_proj_data.rds',
 'date_column': 'ENR_DATE',
 'id_column': 'SUBJECT_ID'}

In [4]:
enrolment_config['data_path'] = '../../impact2_engine/data/' + enrolment_config['data_path']

In [5]:
enrolment_config

{'data_path': '../../impact2_engine/data/enroll_proj_data.rds',
 'date_column': 'ENR_DATE',
 'id_column': 'SUBJECT_ID'}

In [6]:
enr = Enrolment(**enrolment_config)

/Users/robert_pinsight/opt/anaconda3/envs/py311/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/Users/robert_pinsight/opt/anaconda3/envs/py311/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [7]:
enr.preprocess_data()

In [8]:
show_by = 'week'
site = 'SITE'
stratify_by = ['STRATA1']
by_donor = True
max_weeks = 50
target_sample = 50_000
columns_to_use = ['SITE', 'STRATA1']
donor_level = False

In [9]:
metrics = enr.get_enrolment_rate_metrics(site, stratify_by, by_donor, ['p_25', 'min'])

In [10]:
metrics

,SITE,STRATA1,min,p_25,donation_percentage,donation_count,donor_percentage,donor_count
0,448,A,5.0,27.00,0.507236,3645,0.500000,537
1,448,B,1.0,26.50,0.492764,3541,0.500000,537
2,501,A,15.0,46.00,0.504352,3882,0.501112,676
3,501,B,11.0,46.00,0.495648,3815,0.498888,673
4,516,A,16.0,19.00,0.514494,4242,0.502941,513
5,516,B,1.0,19.75,0.485506,4003,0.497059,507


In [11]:
bar_chart = enr.get_bar_chart(show_by=show_by, by_donor=by_donor, stratify_by = stratify_by)

In [12]:
bar_chart.head()

,week,STRATA1,absolute,cumsum,percentage
0,2019-12-30,B,1,1,100.000000
1,2020-01-13,A,113,113,49.561404
2,2020-01-13,B,115,116,50.438596
3,2020-01-20,A,83,196,51.552795
4,2020-01-20,B,78,194,48.447205


In [13]:
model_proj = enr.get_model_projections(max_weeks, target_sample, columns_to_use, donor_level)

17:36:34 - cmdstanpy - INFO - Chain [1] start processing
17:36:35 - cmdstanpy - INFO - Chain [1] done processing
17:36:35 - cmdstanpy - INFO - Chain [1] start processing
17:36:35 - cmdstanpy - INFO - Chain [1] done processing
17:36:35 - cmdstanpy - INFO - Chain [1] start processing
17:36:36 - cmdstanpy - INFO - Chain [1] done processing
17:36:36 - cmdstanpy - INFO - Chain [1] start processing
17:36:36 - cmdstanpy - INFO - Chain [1] done processing
17:36:36 - cmdstanpy - INFO - Chain [1] start processing
17:36:37 - cmdstanpy - INFO - Chain [1] done processing
17:36:37 - cmdstanpy - INFO - Chain [1] start processing
17:36:37 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
model_proj

,day,yhat_lower,yhat_upper,yhat,SITE,STRATA1
0,2020-01-26,132.558772,196.484440,163.806825,516,B
1,2020-01-27,632.652881,697.855766,664.942680,516,B
2,2020-01-28,1178.545369,1243.697119,1210.748553,516,B
3,2020-01-29,1726.189940,1790.348343,1759.525017,516,B
4,2020-01-30,2140.241572,2198.257200,2166.951783,516,B
...,...,...,...,...,...,...
100,2020-05-09,-255.125274,8809.119869,4390.796468,516,B
101,2020-05-10,-475.180195,9030.214337,4402.987013,516,B
102,2020-05-11,-645.523713,9106.497303,4381.268113,516,B
103,2020-05-12,-773.873353,9310.987051,4404.219230,516,B


In [15]:
metrics

,SITE,STRATA1,min,p_25,donation_percentage,donation_count,donor_percentage,donor_count
0,448,A,5.0,27.00,0.507236,3645,0.500000,537
1,448,B,1.0,26.50,0.492764,3541,0.500000,537
2,501,A,15.0,46.00,0.504352,3882,0.501112,676
3,501,B,11.0,46.00,0.495648,3815,0.498888,673
4,516,A,16.0,19.00,0.514494,4242,0.502941,513
5,516,B,1.0,19.75,0.485506,4003,0.497059,507


In [16]:
proj_steps = metrics[columns_to_use + ['p_25']].rename(columns={'p_25': 'proj_value'}).to_dict()

In [17]:
proj = enr.get_projections(max_weeks, target_sample, columns_to_use, donor_level, proj_steps)

In [19]:
proj

,week,SITE,STRATA1,value
0,2019-12-30,448,A,0.0
1,2020-01-13,448,A,1319.0
2,2020-01-20,448,A,1876.0
3,2020-01-27,448,A,2089.0
4,2020-02-03,448,A,2463.0
...,...,...,...,...
67,2020-02-24,516,B,3818.0
68,2020-03-02,516,B,3892.0
69,2020-03-09,516,B,3971.0
70,2020-03-16,516,B,4002.0
